In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import det, inv
from scipy.special import gamma
from math import pi, ceil
from scipy.special import erfinv
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
# from statsmodels.tsa import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from random import randint
import warnings
from sklearn.metrics import mean_squared_error
from math import sqrt
import pymc as pm

warnings.filterwarnings("ignore")

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [9]:
def generate_synthetic_data_df_multivar(n_samples=200, n_exog_vars=4):
    """
    Generates a synthetic dataset with a clear relationship between the target and multiple exogenous variables.
    Returns the data as pandas DataFrames.
    """
    exog_data = {}
    
    for i in range(n_exog_vars):
        # Exogenous variable
        exog_data[i] = np.linspace(0, 10*(i+1), n_samples) + np.random.normal(0, 1, n_samples)
    
    # Create a DataFrame for exogenous variables
    exog_df = pd.DataFrame(exog_data)
    
    # Target variable influenced by the sum of exogenous variables with some noise
    target = exog_df.sum(axis=1) + 5 * np.sin(exog_df[0]) + np.random.normal(0, 2, n_samples)
    target_df = pd.DataFrame(target, columns=[0])
    
    return target_df, exog_df

def plot_data_distributions_grid(target_df, exog_df):
    """
    Plots the distributions of the target and multiple exogenous variables in a grid layout.
    """
    n_exog_vars = exog_df.shape[1]
    
    fig, axes = plt.subplots(nrows=1, ncols=n_exog_vars+1, figsize=(8, 3))

    # Plotting target variable distribution
    target_df[0].hist(ax=axes[0], bins=30, color='blue', alpha=0.3)
    axes[0].set_title("Target Variable Distribution")
    axes[0].set_xlabel("Value")
    axes[0].set_ylabel("Frequency")

    # Plotting exogenous variable distributions
    for i, col in enumerate(exog_df.columns):
        exog_df[col].hist(ax=axes[i+1], bins=30, color='green', alpha=0.3)
        axes[i+1].set_title(f"{col} Distribution")
        axes[i+1].set_xlabel("Value")

    plt.tight_layout()
    plt.show()


# Generate synthetic data with multiple exogenous variables as DataFrames
# synthetic_target_df_multivar, synthetic_exog_df_multivar = generate_synthetic_data_df_multivar()

# Plot distributions for the synthetic data
synthetic_target, synthetic_exog = generate_synthetic_data_df_multivar()
# print(len(ts_train),len(ts_train[0]))#,len(ts_train[0][0]))

In [10]:
def preprocessing(data, method):
    ts_train = []
    ts_valid = []
    ts_test = []
    ts_param = []
    flag_n = False
    flag_diff = False
    flag_dec = False
    if method is 'normalization':
        flag_n = True
    elif method is 'differencing':
        flag_diff = True
            
    for i in range(data.shape[1]):

        temp = list(data.iloc[6:][i].dropna())
        # print(temp)
        if len(temp) > 130:
            
            cut_off_1 = ceil(len(temp)*0.7)
            cut_off_2 = ceil(len(temp)*0.9)

            temp_train = temp[:cut_off_1]
            temp_val = temp[cut_off_1:cut_off_2]
            temp_test = temp[cut_off_2:]

            if flag_n is True:
                ts_param.append([np.mean(temp_train), np.std(temp_train), np.mean(temp_val), 
                np.std(temp_val), np.mean(temp_test), np.std(temp_test)])
                
                temp_train = (temp_train - np.mean(temp_train)) / np.std(temp_train)
                ts_train.append(temp_train)
                

                temp_val = (temp_val - np.mean(temp_val)) / np.std(temp_val)
                ts_valid.append(temp_val)

                temp_test = (temp_test - np.mean(temp_test)) / np.std(temp_test)
                ts_test.append(temp_test)
                

            elif flag_diff is True:

                temp_train = pd.Series(temp_train)
                temp_train_log_diff = temp_train - temp_train.shift()
                temp_train_log_diff[0] = temp_train[0]
                temp_train_log_diff.dropna(inplace=True)
                ts_train.append(temp_train_log_diff.values)

                temp_val = pd.Series(temp_val)
                temp_val_log_diff = temp_val - temp_val.shift()
                temp_val_log_diff[0] = temp_val[0]
                temp_val_log_diff.dropna(inplace=True)
                ts_valid.append(temp_val_log_diff.values)

                temp_test = pd.Series(temp_test)
                temp_test_log_diff = temp_test - temp_test.shift()
                temp_test_log_diff[0] = temp_test[0]
                temp_test_log_diff.dropna(inplace=True)
                ts_test.append(temp_test_log_diff.values)
                
    return ts_train, ts_valid, ts_test, ts_param

# ts_train, ts_valid, ts_test, ts_param = preprocessing(df_temp, method='normalization')

In [138]:
class AutoregressiveTree:
    
    def __init__(self, p, u0=0, alpha_u=1, X=None):
        self._X = X  # Exogenous data
        
        erf_temp = np.zeros([7,1])
        for i in range(1,8):
            erf_temp[i-1] = erfinv((i/4) - 1)
        
        self._erf = erf_temp
        self._p = p
        self._alpha_W = p + 2

        self._u0 = u0
        self._alpha_u = alpha_u
        self.target = []
        self.exog = []
    #  calculate the sample mean of the data (could be a vector), maybe split into sample mean of each variable
    def sample_mean(self, data):
        # print(sum(data), len(data))
        return sum(np.asarray(data))/len(data)

    
    # calculate the scatter matrix around the mean uN
    # def scatter_matrix(self, data, uN_):
    def scatter_matrix(self, data, uN_):

        # assert data.shape[1] == p * (1 + no_exog_vars), 'Data dimensions do not match expected shape'
        # Assuming data has been preprocessed to include lags of y and X
        temp = data - uN_

        SN = 0
        for row in temp:
            row = row[:, np.newaxis]
            SN += row * row.T
        # print(len(SN),len(SN[0]))
        return SN
    # def WN_func(self, uN_, SN, W0, N):
    def WN_func(self, uN_, SN, W0, N):
        # assert SN.shape[0] == p * (1 + no_exog_vars), 'Scatter matrix dimensions do not match expected shape'
        temp = self._u0 - uN_

        # Assuming scatter matrix has been computed from preprocessed data
        temp = temp[:, np.newaxis]
        WN = W0 + SN + ((self._alpha_u * N) / (self._alpha_u + N)) * np.dot(temp, temp.T)
        return WN
    # Updates the matrix WN_d, calculates the within node covariance matrix 
    def WN_d_func(self, u0_, uN_d_, SN_d_, W0_, N_):
        temp = u0_ - uN_d_
        temp = temp[:, np.newaxis]
        WN_ = W0_ + SN_d_ + ((self._alpha_u * N_) / (self._alpha_u + N_)) * np.dot(temp, temp.T)
        return WN_
    # calculating the Maximum a posteriori arameters for the ar model
    def MAP_param(self, N, uN_, WN):
        ut = ((self._alpha_u * self._u0) + (N * uN_)) / (self._alpha_u + N)
        Wt_inv = (1 / (self._alpha_W + N - (self._p + 1))) * WN
        return ut, Wt_inv
    # calculate all of the AR parameters needed 
    def param(self, target, exog):
        # exog = np.array(exog)
        # exog2 = np.reshape(exog.transpose(1, 0, 2), (len(exog[0]), len(exog)*len(exog[0][0])))
        # data = np.hstack((target, exog2))
        self.target, self.exog = target, exog
        
        data = []
        for t, e in zip(target, np.asarray(exog).transpose(1, 0, 2)):
                data.append(np.concatenate([t, e.flatten()]))
        
        data = np.array(data)
        y = data[:, 0]
        y_lags = data[:, 1:self._p + 1]
        x_lags = data[:, self._p + 1:]
        
        with pm.Model() as arx_model:
            # Priors
            intercept = pm.Normal('intercept', mu=0, sigma=10)
            phi = pm.Normal('phi', mu=0, sigma=10, shape=self._p)
            theta = pm.Normal('theta', mu=0, sigma=10, shape=data.shape[1]-self._p-1)
            sigma = pm.HalfNormal('sigma', sigma=10)
            
            # Expected value
            mu = intercept + pm.math.dot(y_lags, phi) + pm.math.dot(x_lags, theta)
            
            # Likelihood
            y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)
            
            # Compute the MAP estimate
            map_estimate = pm.find_MAP(progressbar=False)

        # m = map_estimate['intercept']
        # b = np.append(map_estimate['phi'], map_estimate['theta'])
        # var = map_estimate['sigma']
        
        # Concatenate list1 and reshaped list2
        # print("datalen:", np.asarray(data).shape)
        N = len(data)
        uN_ = self.sample_mean(data)
        SN = self.scatter_matrix(data, uN_)
        W0 = np.identity(SN.shape[0])
        WN = self.WN_func(uN_, SN, W0, N)
        ut, Wt_inv = self.MAP_param(N, uN_, WN)
        W = inv(Wt_inv)
        var = 1 / W[-1, -1]
        Wpp = inv(Wt_inv[:-1, :-1])
        # print(Wpp.shape)
        b = np.zeros([len(data[0]), 1])
        for j in range(len(b)-1):
            for i in range(self._p):
                b[j] += Wt_inv[-1, i] * Wpp[i, j]
        b = b[:-1]
        m = ut[-1]
        for i in range(len(b)):
            m += b[i] * ut[i]
        
        return var, b, m[0]
    # scaling function using the gamma distribution
    def c_func(self, l, alpha):
        c = 1
        #   for loop goes from 1 to l
        for i in range(1, l + 1):
            c *= gamma((alpha + 1 - i) / 2)
        
        return c
    # probability density scaling function used
    def pds_func(self, N, W0, WN):
        pds = (pi**(-((self._p + 1) * N) / 2)) + \
        ((self._alpha_u / (self._alpha_u + N))**((self._p + 1) / 2)) + \
        (self.c_func(self._p + 1, self._alpha_W + N) / self.c_func(self._p + 1, self._alpha_W)) * (det(W0)**(self._alpha_W / 2))*(det(WN)**(-(self._alpha_W + N) / 2))
        return pds
    # similiar to above just now with different params
    def pd_s_func(self, u0_, N_, W0_, WN_):
        pds = (pi**(-((self._p + 1) * N_) / 2)) + \
        ((self._alpha_u / (self._alpha_u + N_))**((self._p + 1) / 2)) + \
        (self.c_func(self._p + 1, self._alpha_W - 1 + N_) / self.c_func(self._p + 1, self._alpha_W - 1)) * (det(W0_)**((self._alpha_W - 1) / 2))*(det(WN_)**(-(self._alpha_W - 1 + N_) / 2))
        return pds
    def mult_func(self, l, alpha, N):
        c = 1
        #   for loop goes from 1 to l
        for i in range(1, l + 1):
            c *= ((alpha + 1 + N - i)/(alpha + 1 - i))
        
        return c

    def pds_func2(self, N, W0, WN, u0_, N_, W0_, WN_):

        pds = (det(W0)**(self._alpha_W / 2))*det(WN)**(-(self._alpha_W + N) / 2) / (det(W0_)**((self._alpha_W - 1) / 2))*(det(WN_)**(-(self._alpha_W - 1 + N_) / 2)) * \
        self.mult_func(self._p + 1,self._alpha_W, N)                                                                                                             

        return pds

    def LeafScore(self, data):
        N = len(data)
        self.target = data
        uN_ = self.sample_mean(data)
        SN = self.scatter_matrix(data, uN_)
        W0 = np.identity(SN.shape[0])
        WN = self.WN_func(uN_, SN, W0, N)
        ut, Wt_inv = self.MAP_param(N, uN_, WN)
        data_ = []
        for x in data:
            data_.append(x[:-1])
        N_ = len(data_)
        uN_d_ = self.sample_mean(data_)
        SN_d_ = self.scatter_matrix(data_, uN_d_)
        u0_ = ut[:-1]
        W0_ = inv(inv(W0)[:-1, :-1])
        WN_d_ = self.WN_d_func(u0_, uN_d_, SN_d_, W0_, N_)

        if N > 20:
            pds2 = self.pds_func2(N, W0, WN, u0_, N_, W0_, WN_d_)
        else:
            pds = self.pds_func(N, W0, WN)
            pds_ = self.pd_s_func(u0_, N_, W0_, WN_d_)
            pds2 =  pds/pds_
        # print("frac: ", pds/pds_)
        # print("frac2: " , pds2)
        return pds2
        
    # This will spplit a dataset into two froups based on the specific features. Then splits the data points into the left or right set
    def test_split(self, index, value, dataset):
        left, right = list(), list()
        for row in dataset:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return left, right
    def rest_split(self, index, value, train, exog, ex, var):
                # Initializations
        left, right = [], []
        left_e, right_e = [], []

        # Convert to DataFrame for easier operations
        train_df = pd.DataFrame(train)
        exog_dfs = [pd.DataFrame(ex) for ex in exog]

        # Split based on exogenous variable
        if ex == "y":
            # Split the specified variable in exog
            for i, row in exog_dfs[var].iterrows():
                if row[index] < value:
                    left_e.append(i)
                else:
                    right_e.append(i)

            left_e_index = pd.Index(left_e)
            right_e_index = pd.Index(right_e)

            # Extract corresponding rows from other variables in exog and from train
            left_exog = [df.loc[left_e_index] for df in exog_dfs]
            right_exog = [df.loc[right_e_index] for df in exog_dfs]

            left = train_df.loc[left_e_index]
            right = train_df.loc[right_e_index]

        # Split based on training data
        else:
            for i, row in train_df.iterrows():
                if row[index] < value:
                    left.append(i)
                else:
                    right.append(i)
            
            left_index = pd.Index(left)
            right_index = pd.Index(right)

            left = train_df.loc[left_index]
            right = train_df.loc[right_index]

            left_exog = [df.loc[left_index] for df in exog_dfs]
            right_exog = [df.loc[right_index] for df in exog_dfs]
        left = left.values.tolist()
        right = right.values.tolist()
        left_exog = [df.values.tolist() for df in left_exog]
        right_exog = [df.values.tolist() for df in right_exog]
        return left, right, left_exog, right_exog
    # itrates through the features and the values to det the best for splitting the dataset, calkculates the score for each split and choses the one with best improvement
    def get_split(self, train, train_exog):
        b_index, b_value, b_groups, var = 999, 999, None, 'y'
        b_score = self.LeafScore(train)
        avg = np.mean(train, axis=0)[:-1]
        sigma = np.std(train, axis=0)[:-1]
        split_data = train, train_exog
        for index in range(len(avg)):
            for i in range(len(self._erf)):

                value = avg[index] + sigma[index] * self._erf[i]
                groups = self.test_split(index, value, train)
                data = self.rest_split(index, value, train, train_exog,"n", 0)
                
                new_score = 1
                for group in groups:
                    if len(group) != 0:
                        new_score *= self.LeafScore(group)
            
                        if new_score > b_score:
                            b_index, b_value, b_score, b_groups, var, split_data = index, value, new_score, groups, ("y"+str(index)), data
        j = 0
        bb_score = 1
        for ex in train_exog:
            # print(j)
            avg = np.mean(ex, axis=0)[:-1]
            bb_score =  max(bb_score, self.LeafScore(ex))
            # print(avg)
            sigma = np.std(ex, axis=0)[:-1]
            for index in range(len(avg)):
                for i in range(len(self._erf)):
                    value = avg[index] + sigma[index] * self._erf[i]
                    groups = self.test_split(index, value, ex)
                    data = self.rest_split(index, value, train, train_exog,"y", j)
                    new_score = 1
                    for group in groups:
                        if len(group) != 0:
                            new_score *= self.LeafScore(group)
                
                            if new_score > b_score:
                                b_index, b_value, b_score, b_groups, var, split_data = index, value, new_score, groups, ("x"+str(j)+str(index)), data
            j += 1
        # print({'index':b_index, 'variable': var, 'value':b_value, 'groups':b_groups})
        # print(var, b_score, bb_score)
        # print(np.asarray(data).shape)
        # self.exog = exog
        return {'index':b_index, 'variable': var, 'value':b_value, 'groups':b_groups, 'split_data':split_data}
    # turns a group of points, belonging to one datagroup into a terminal node, calculates the parameters for that specific group
    def to_terminal(self, target, exog):
        outcomes = self.param2(target,exog)
        return outcomes
    # this recursivelky builds the tree up. If the node should be a terminal node make terminal, else use get split to find next best split
    def split(self, node, max_depth, min_size, depth):
        left, right = node['groups']
        left_t, right_t, left_e, right_e = node['split_data']
        del(node['groups'])
        if not left or not right:
            node['left'] = node['right'] = self.to_terminal(left_t + right_t, left_e + right_e)
            return
        
        if depth >= max_depth:
            node['left'], node['right'] = self.to_terminal(left_t,left_e ), self.to_terminal(right_t, right_e)
            return
        
        if len(left) <= min_size:
            node['left'] = self.to_terminal(left_t, left_e)
        else:
            node['left'] = self.get_split(left_t, left_e)
            if node['left']['groups'] is None:
                node['left'] = self.to_terminal(left_t, left_e)
            else:
                self.split(node['left'], max_depth, min_size, depth+1)
        
        if len(right) <= min_size:
            node['right'] = self.to_terminal(right_t, right_e)
        else:
            node['right'] = self.get_split(right_t, right_e)
            if node['right']['groups'] is None:
                node['right'] = self.to_terminal(right_t, right_e)
            else:
                self.split(node['right'], max_depth, min_size, depth+1)
    # initiates the buiilding process. Finds initial split and if there are no effective splits then makes source node a terminal node
    def build_tree(self, train, train_exog, max_depth, min_size):
        train=train
        train_exog = train_exog
        root = self.get_split(train, train_exog)
        if root['groups'] is None:
            root['root'] = self.to_terminal(train,train_exog)
            root['index'] = None
            root['value'] = None
            del(root['groups'])
        else:
            # print(root['index'])
            self.split(root, max_depth, min_size, 1)
        
        return root
    # prints the tree structure 
    def print_tree(self, node, depth=0):
        if isinstance(node, dict):
            if node['value'] is None:
                print(node)
                return                                                                                                                               
            print('%s[%s < %.3f]' % ((depth*' ', (node['variable']), node['value'])))
            # print(depth)
            # print(node)
            self.print_tree(node['left'], depth+1)
            self.print_tree(node['right'], depth+1)
    
        else:
            print('%s[%s]' % ((depth*' ', node)))
    # follows the tree strarting from root node until a terminal node is found
    def predict(self, node, row):
        # Expecting the input row to include lags of both y and X
        if 'root' in node:
            return node['root']
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict):
                return self.predict(node['left'], row)
            else:
                return node['left']
        else:
            if isinstance(node['right'], dict):
                return self.predict(node['right'], row)
            else:
                return node['right']
    def param2(self, target, exog):
        self.target, self.exog = target, exog
    
    # Constructing data similar to your earlier approach
        data = []
        for t, e in zip(target, np.asarray(exog).transpose(1, 0, 2)):
            data.append(np.concatenate([t, e.flatten()]))
        data = np.array(data)
        
        # Estimation for ARX parameters
        N = data.shape[0]
        uN_ = self.sample_mean(data)
        SN = self.scatter_matrix(data, uN_)
        W0 = np.identity(SN.shape[0])
        WN = self.WN_func(uN_, SN, W0, N)
        ut, Wt_inv = self.MAP_param(N, uN_, WN)
        W = inv(Wt_inv)
        
        var = 1 / W[-1, -1]
        Wpp = inv(Wt_inv[:-1, :-1])
        b = np.zeros([self._p, 1])
        for j in range(SN.shape[0]):
            for i in range(self._p):
                b[j] += Wt_inv[-1, i] * Wpp[i, j]
        
        m = ut[-1]
        for i in range(SN.shape[0]):
            m += b[i] * ut[i]
        
        return var, b, m



In [124]:
import numpy as np
import pandas as pd
from numpy.linalg import inv, det
from scipy.special import gamma, erfinv
from math import pi, ceil

class AutoregressiveTree:
    
    def __init__(self, p, u0=0, alpha_u=1, X=None):
        self._X = X  # Exogenous data
        
        erf_temp = np.zeros([7,1])
        for i in range(1,8):
            erf_temp[i-1] = erfinv((i/4) - 1)
        
        self._erf = erf_temp
        self._p = p
        self._alpha_W = p + 2
        self._u0 = u0
        self._alpha_u = alpha_u
        self.target = []
        self.exog = []

    def sample_mean(self, data):
        return sum(np.asarray(data))/len(data)

    def scatter_matrix(self, data, uN_):
        temp = data - uN_
        SN = 0
        for row in temp:
            row = row[:, np.newaxis]
            SN += row * row.T
        return SN

    def WN_func(self, uN_, SN, W0, N):
        temp = self._u0 - uN_
        temp = temp[:, np.newaxis]
        WN = W0 + SN + ((self._alpha_u * N) / (self._alpha_u + N)) * np.dot(temp, temp.T)
        return WN

    def WN_d_func(self, u0_, uN_d_, SN_d_, W0_, N_):
        temp = u0_ - uN_d_
        temp = temp[:, np.newaxis]
        WN_ = W0_ + SN_d_ + ((self._alpha_u * N_) / (self._alpha_u + N_)) * np.dot(temp, temp.T)
        return WN_

    def MAP_param(self, N, uN_, WN):
        ut = ((self._alpha_u * self._u0) + (N * uN_)) / (self._alpha_u + N)
        Wt_inv = (1 / (self._alpha_W + N - (self._p + 1))) * WN
        return ut, Wt_inv

    def param(self, target, exog):
        self.target, self.exog = target, exog
        data = []
        for t, e in zip(target, np.asarray(exog).transpose(1, 0, 2)):
            data.append(np.concatenate([t, e.flatten()]))
        data = np.asarray(data)
        N = len(data)
        uN_ = self.sample_mean(data)
        SN = self.scatter_matrix(data, uN_)
        W0 = np.identity(SN.shape[0])
        WN = self.WN_func(uN_, SN, W0, N)
        ut, Wt_inv = self.MAP_param(N, uN_, WN)
        W = inv(Wt_inv)
        var = 1 / W[-1, -1]
        Wpp = inv(Wt_inv[:-1, :-1])
        b = np.zeros([self._p, 1])
        for j in range(len(b)):
            for i in range(self._p):
                b[j] += Wt_inv[-1, i] * Wpp[i, j]
        m = ut[-1]
        for i in range(self._p):
            m += b[i] * ut[i]
        return var, b, m[0]

    def c_func(self, l, alpha):
        c = 1
        for i in range(1, l + 1):
            c *= gamma((alpha + 1 - i) / 2)
        return c

    def pds_func(self, N, W0, WN):
        pds = (pi**(-((self._p + 1) * N) / 2)) + \
        ((self._alpha_u / (self._alpha_u + N))**((self._p + 1) / 2)) + \
        (self.c_func(self._p + 1, self._alpha_W + N) / self.c_func(self._p + 1, self._alpha_W)) * (det(W0)**(self._alpha_W / 2))*(det(WN)**(-(self._alpha_W + N) / 2))
        return pds

    def pd_s_func(self, N_, W0_, WN_):
        pds = (pi**(-((self._p + 1) * N_) / 2)) + \
        ((self._alpha_u / (self._alpha_u + N_))**((self._p + 1) / 2)) + \
        (self.c_func(self._p + 1, self._alpha_W - 1 + N_) / self.c_func(self._p + 1, self._alpha_W - 1)) * (det(W0_)**((self._alpha_W - 1) / 2))*(det(WN_)**(-(self._alpha_W - 1 + N_) / 2))
        return pds

    def mult_func(self, l, alpha, N):
        c = 1
        for i in range(1, l + 1):
            c *= ((alpha + 1 + N - i)/(alpha + 1 - i))
        return c

    def pds_func2(self, N, W0, WN, N_, W0_, WN_):
        pds = (det(W0)**(self._alpha_W / 2))*det(WN)**(-(self._alpha_W + N) / 2) / (det(W0_)**((self._alpha_W - 1) / 2))*(det(WN_)**(-(self._alpha_W - 1 + N_) / 2)) * \
        self.mult_func(self._p + 1,self._alpha_W, N)
        return pds

    def pds_func2_singular(self, N, W0, WN, N_, W0_, WN_):
        if np.isscalar(WN) or WN.ndim == 0:
            det_WN = WN
        else:
            det_WN = det(WN)
        if np.isscalar(WN_) or WN_.ndim == 0:
            det_WN_ = WN_
        else:
            det_WN_ = det(WN_)
        pds = ((W0)**(self._alpha_W / 2))*det_WN**(-(self._alpha_W + N) / 2) / ((W0_)**((self._alpha_W - 1) / 2))*(det_WN_**(-(self._alpha_W - 1 + N_) / 2)) * \
        self.mult_func(self._p + 1,self._alpha_W, N)
        return pds

    def LeafScore(self, data):
        self.exog = data
        N = len(data)
        exog = False
        data_ = []
        try:
            check = (len(data[0]) > 2)
        except Exception as e:
            print(e)
            print("EXOG")
            exog = True       

        if exog:
            data_ = (data[:-1])
            N_ = len(data_)
            uN_ = self.sample_mean(data)
            uN_d_ = self.sample_mean(data_)
            SN = np.std(data)
            W0 = 1
            temp = np.asarray(-uN_)
            WN = 1 + 1 + ((self._alpha_u * N) / (self._alpha_u + N)) * np.dot(temp, temp.T)
            SN = np.std(data_)
            W0_ = 1
            temp = np.asarray(-uN_d_)
            WN_d_ = 1 + 1 + ((self._alpha_u * N_) / (self._alpha_u + N_)) * np.dot(temp, temp.T)
        else:
            for x in data:
                data_.append(x[:-1])
            N_ = len(data_)
            uN_ = self.sample_mean(data)
            uN_d_ = self.sample_mean(data_)
            SN = self.scatter_matrix(data, uN_)
            W0 = np.identity(SN.shape[0])
            WN = self.WN_func(uN_, SN, W0, N)
            SN_d_ = self.scatter_matrix(data_, uN_d_)
            W0_ = inv(inv(W0)[:-1, :-1])
            ut, Wt_inv = self.MAP_param(N, uN_, WN)
            u0_ = ut[:-1]
            WN_d_ = self.WN_d_func(u0_, uN_d_, SN_d_, W0_, N_)

        if N > 20:
            if exog:
                pds2 = self.pds_func2_singular(N, W0, WN, N_, W0_, WN_d_)
            else:
                pds2 = self.pds_func2(N, W0, WN, N_, W0_, WN_d_)
        else:
            pds = self.pds_func(N, W0, WN)
            pds_ = self.pd_s_func(N_, W0_, WN_d_)
            pds2 =  pds/pds_

        return pds2

    def test_split(self, index, value, dataset):
        exog = False
        try:
            check = isinstance(len(dataset[0]), list)
        except Exception as e:
            print(e)
            print("EXOG")
            exog = True    
        if exog == False:
            left, right = list(), list()
            for row in dataset:
                if row[index] < value:
                    left.append(row)
                else:
                    right.append(row)
        else:
            left, right = list(), list()
            for row in dataset:
                if row < value:
                    left.append(row)
                else:
                    right.append(row)
        return left, right

    def rest_split(self, index, value, train, exog, ex, var):
        left, right = [], []
        left_e, right_e = [], []

        train_df = pd.DataFrame(train)
        exog_dfs = [pd.DataFrame(ex) for ex in exog]

        if ex == "y":
            for row in exog_dfs[var]:
                if row < value:
                    left_e.append(row)
                else:
                    right_e.append(row)
            left_e_index = pd.Index(left_e)
            right_e_index = pd.Index(right_e)

            left_exog = [df.loc[left_e_index] for df in exog_dfs]
            right_exog = []
            for df in exog_dfs:
                try:
                    right_exog.append(df.loc[right_e_index])
                except KeyError as e:
                    print(f"Error with DataFrame: \n{df}\nError: {e}")
            left = train_df.loc[left_e_index]
            right = train_df.loc[right_e_index]
        else:
            for i, row in train_df.iterrows():
                if row[index] < value:
                    left.append(i)
                else:
                    right.append(i)
            
            left_index = pd.Index(left)
            right_index = pd.Index(right)

            left = train_df.loc[left_index]
            right = train_df.loc[right_index]

            left_exog = [df.loc[left_index] for df in exog_dfs]
            right_exog = [df.loc[right_index] for df in exog_dfs]

        left = left.values.tolist()
        right = right.values.tolist()
        left_exog = [df.values.tolist() for df in left_exog]
        right_exog = [df.values.tolist() for df in right_exog]
        return left, right, left_exog, right_exog

    def get_split(self, train, train_exog):
        b_index, b_value, b_groups, var = 999, 999, None, 'y'
        b_score = self.LeafScore(train)
        avg = np.mean(train, axis=0)[:-1]
        sigma = np.std(train, axis=0)[:-1]
        split_data = train, train_exog
        for index in range(len(avg)):
            for i in range(len(self._erf)):
                value = avg[index] + sigma[index] * self._erf[i]
                groups = self.test_split(index, value, train)
                data = self.rest_split(index, value, train, train_exog, 'x', var)
                gini = self.LeafScore(data[0])
                if gini < b_score:
                    b_index, b_value, b_score, b_groups, var = index, value, gini, data, 'x'
        if not isinstance(train_exog[0], list):
            train_exog = [train_exog]
        exog_avg = [np.mean(exog, axis=0) for exog in train_exog]
        exog_sigma = [np.std(exog, axis=0) for exog in train_exog]
        for j, ex in enumerate(train_exog):
            for index in range(len(exog_avg[j])):
                for i in range(len(self._erf)):
                    value = exog_avg[j][index] + exog_sigma[j][index] * self._erf[i]
                    groups = self.test_split(index, value, ex)
                    data = self.rest_split(index, value, train, train_exog, 'y', j)
                    gini = self.LeafScore(data[0])
                    if gini < b_score:
                        b_index, b_value, b_score, b_groups, var = index, value, gini, data, j
        return {'index':b_index, 'value':b_value, 'groups':b_groups, 'var':var}
    def to_terminal(self, target, exog):
        outcomes = self.param(target, exog)
        return outcomes

    def split(self, node, max_depth, min_size, depth):
        left, right = node['groups']
        del(node['groups'])

        if not left or not right:
            node['left'] = node['right'] = self.to_terminal(left + right, [])
            return

        if depth >= max_depth:
            node['left'], node['right'] = self.to_terminal(left, []), self.to_terminal(right, [])
            return

        if len(left) <= min_size:
            node['left'] = self.to_terminal(left, [])
        else:
            node['left'] = self.get_split(left, [])
            self.split(node['left'], max_depth, min_size, depth+1)

        if len(right) <= min_size:
            node['right'] = self.to_terminal(right, [])
        else:
            node['right'] = self.get_split(right, [])
            self.split(node['right'], max_depth, min_size, depth+1)

    def build_tree(self, train, train_exog, max_depth, min_size):
        root = self.get_split(train, train_exog)
        self.split(root, max_depth, min_size, 1)
        return root

    def print_tree(self, node, depth=0):
        if isinstance(node, dict):
            if node['value'] is None:
                print(node)
                return                                                                                                                               
            print('%s[%s < %.3f]' % ((depth*' ', (node['variable']), node['value'])))
            self.print_tree(node['left'], depth+1)
            self.print_tree(node['right'], depth+1)
        else:
            print('%s[%s]' % ((depth*' ', node)))

    def predict(self, node, row):
        if 'root' in node:
            return node['root']
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict):
                return self.predict(node['left'], row)
            else:
                return node['left']
        else:
            if isinstance(node['right'], dict):
                return self.predict(node['right'], row)
            else:
                return node['right']


In [149]:
p = 3
df_temp2 = pd.concat([synthetic_target,synthetic_exog], axis=1)
df_temp2.columns = range(df_temp2.shape[1])

ts_train, ts_valid, ts_test, ts_param = preprocessing(df_temp2, method='normalization')

train = []
for s in ts_train:
    temp = []
    for i in range(len(s) - (p + 1)):
        temp.append(s[i:i + p + 1])


    train.append(temp)
d = train[0]
# print(len(d), len(d[0]))
d_exog = ts_train[1:]
# d_exog = d_exog.T
ART = AutoregressiveTree(3)
tree = ART.build_tree(d, synthetic_exog.to_numpy(), 10, 5)

ART.print_tree(tree)


KeyError: '[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 29, 30, 32, 36, 37] not in index'

In [148]:
synthetic_exog.to_numpy().shape

(200, 4)

In [75]:
print(len(arr), len(arr[0]))

132 4
